In [2]:
import os
import glob
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('max_rows', 300)
pd.set_option('max_columns', 200)

perc = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]

readPath = r'\\192.168.10.28\equityTradeLogs'
dataPathLs = np.array(glob.glob(os.path.join(readPath, 'speedCompare***.csv')))
path = np.sort(dataPathLs)[-12:-1]
cur = np.max([int(os.path.basename(i).split('_')[1].split('.')[0]) for i in path])
assert(cur == 20201207)


body = "<html><body><div>" + 'Hi all,<p>The following is daily report based on ' + str(cur) + ' data.</p>'
body += '<p><b>I. Assertions</p></b>'
count = 0

# load data
print('-----------------------------------------------------------------------------------------------')
print('load data')
rawOrderLog = []
for thisPath in path:
    data = pd.read_csv(thisPath)
    data = data.rename(columns={'mdClockAtArrival': 'caamd'})
    rawOrderLog += [data]
rawOrderLog = pd.concat(rawOrderLog, sort=False)

for col in ['clockAtArrival', 'caamd', 'secid', 'updateType', 'vai', 'absFilledThisUpdate', 'orderDirection', 'absOrderSize',
            'absOrderSizeCumFilled', 'date', 'accCode', 'mse']:
    rawOrderLog[col] = rawOrderLog[col].fillna(0).astype('int64')   
rawOrderLog = rawOrderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)

rawOrderLog = rawOrderLog[rawOrderLog["secid"] >= 1000000]

if rawOrderLog[rawOrderLog.duplicated(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                    'orderDirection', 'absOrderSize'], keep=False)].shape[0] != 0:
    display('There are accounts with duplicated ticks:')
    display(rawOrderLog[rawOrderLog.duplicated(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                    'orderDirection', 'absOrderSize'], keep=False)]\
.groupby(['date', 'colo', 'accCode'])['ars'].size())
    rawOrderLog = rawOrderLog.drop_duplicates(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                    'orderDirection', 'absOrderSize'], keep='first')

display('There are ticks with orderDirection 0')
display(rawOrderLog[rawOrderLog['orderDirection'] == 0][['date', 'colo', 'accCode', \
            'secid', 'vai', 'updateType', 'sdd', 'orderDirection', 'absOrderSize', 'internalId', 'orderId']])
try:
    num1 = rawOrderLog[(rawOrderLog['orderDirection'] == 0) & (rawOrderLog['date'] == cur)].shape[0]
    assert(num1 < 30)
except:
    count += 1
    body += str(count) + '. There are in total ' + str(num1) + ' orders with orderDirection 0.<div>'



assert(rawOrderLog[rawOrderLog['updateType'] == 0][rawOrderLog[rawOrderLog['updateType'] == 0]\
                                                   .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                'vai', 'absOrderSize', 'internalId'], keep=False)].shape[0] == 0)
try:
    assert(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                       .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                    'absOrderSize', 'internalId'], keep=False)].shape[0] == 0)
except:
    print('There are orders with all things same except sdd')
    print(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                       .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                    'absOrderSize', 'internalId'], keep=False)])
    assert(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                       .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                    'absOrderSize', 'internalId', 'sdd'], keep=False)].shape[0] == 0)
try:
    assert(sum(rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique() != 1) == 0) 
except:
    print('There are orders with same internalId but different orderId other than accCode 8856 case')
    print(rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique()[rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique() > 1])

r2 = rawOrderLog[(rawOrderLog['accCode'] != 8856) & (rawOrderLog['orderDirection'] != 0)]
r1 = rawOrderLog[(rawOrderLog['accCode'] == 8856) & (rawOrderLog['orderDirection'] != 0)]
r1['test'] = r1.groupby(['date', 'colo', 'accCode', 'secid', 
            'orderDirection', 'absOrderSize']).grouper.group_info[0]
r1 = r1.sort_values(by=['test', 'clockAtArrival'])
r1.loc[r1['updateType'] != 0, 'vai'] = np.nan
r1['vai'] = r1.groupby('test')['vai'].ffill()
r2['test'] = r2.groupby(['date', 'colo', 'accCode', 'secid', 
            'orderDirection', 'absOrderSize', 'internalId']).grouper.group_info[0]
r2 = r2.sort_values(by=['test', 'clockAtArrival'])
r2.loc[r2['updateType'] != 0, 'vai'] = np.nan
r2['vai'] = r2.groupby('test')['vai'].ffill()
assert(sum(r1[r1['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1) == 0)
try:
    assert(sum(r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1) == 0)
except:
    a = r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique()[r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1].reset_index()
    print(pd.merge(r2, a[['test', 'vai']], on=['test', 'vai'], how='inner')[['secid', 'accCode', 'colo', 'vai', 'updateType', 'sdd', 'internalId', 'orderId', 'absOrderSize', 'absFilledThisUpdate', 'absOrderSizeCumFilled', 'orderPrice', 'tradePrice']])
orderLog = pd.concat([r1, r2])
del r1
del r2    

orderLog['clock'] = orderLog['clockAtArrival'].apply(lambda x: datetime.datetime.fromtimestamp(x/1e6))
orderLog['broker'] = orderLog['accCode'] // 100
orderLog["broker"] = np.where(orderLog["accCode"].astype(str).apply(lambda x: len(x) == 6), orderLog['accCode'] // 10000, orderLog["broker"])
orderLog['colo_broker'] = orderLog['colo'].str[:2] + '_' + orderLog['broker'].astype('str')
orderLog['order'] = orderLog.groupby(['date', 'colo', 'accCode', 'secid', 'vai']).grouper.group_info[0]
orderLog['group'] = orderLog.groupby(['date', 'secid', 'vai']).grouper.group_info[0]
orderLog['startClock'] = orderLog.groupby(['order'])['clockAtArrival'].transform('first')
orderLog['duration'] = orderLog['clockAtArrival'] - orderLog['startClock']
orderLog['orderPrice'] = orderLog['orderPrice'].apply(lambda x: round(x, 2))
orderLog['tradePrice'] = orderLog['tradePrice'].apply(lambda x: round(x, 2))
orderLog['orderDirection1'] = np.where(orderLog["orderDirection"] == -2, -1, np.where(
    orderLog["orderDirection"] == 2, 1, orderLog["orderDirection"]))
orderLog["ars"] = orderLog.groupby(['order'])['ars'].transform('first')


### Assertion 1:  make sure same direction in same date, secid, vai
print('=======================================================================================')
print('1. same date, secid, vai: same direction')
orderLog['directNum'] = orderLog.groupby(['date', 'secid', 'vai'])['orderDirection1'].transform('nunique')
if len(orderLog[orderLog['directNum'] != 1]) > 0:
    print('opposite direction for same date, same secid, same vai')
    display(orderLog[(orderLog['directNum'] != 1) & (orderLog['updateType'] == 0)].groupby(['date'])['orderDirection'].size())
    display(orderLog[(orderLog['directNum'] != 1) & (orderLog['updateType'] == 0)].groupby(['date', 'accCode'])['orderDirection'].size())
    try:
        num1 = orderLog[(orderLog['directNum'] != 1) & (orderLog['updateType'] == 0) & (orderLog['date'] == cur)].shape[0]
        assert(num1 < 200)
    except:
        count += 1
        body += str(count) + '. There are in total ' + str(num1) + ' orders with opposite directions under same date, secid, vai.<div>'
    try:
        num2 = list(orderLog[(orderLog['directNum'] != 1) & (orderLog['updateType'] == 0) & (orderLog['date'] == \
            cur)].groupby('accCode')['date'].size()[orderLog[(orderLog['directNum'] != 1) & \
           (orderLog['updateType'] == 0) & (orderLog['date'] == cur)].\
                                                                       groupby('accCode')['date'].size() > 50].index)
        assert(len(num2) == 0)
    except:
        count += 1
        num2 = ', '.join([str(x) for x in num2])
        body += str(count) + '. ' + num2 + ' has more than 50 orders with opposite directions under same date, secid, vai.<div>'
    orderLog = orderLog[orderLog['directNum'] == 1]

assert((orderLog.groupby(['date', 'secid', 'vai'])['orderDirection1'].nunique() == 1).all() == True)

## Assertion 2:  make sure each account, secid, vai only has one insertion
print('=======================================================================================')
print('2. same date, secid, vai, accCode: one insertion')
a = orderLog[orderLog['updateType'] == 0].groupby(['date', 'accCode', 'secid', 'vai', 'order'])['clockAtArrival'].count()
if len(a[a > 1]) > 0:
    print('more than one insertion at same time')
    a = a[a>1].reset_index()
    display(a)
    display(a.groupby(['date'])['accCode'].size())
    display(a.groupby(['date', 'accCode'])['order'].size())
    try:
        num1 = a[a['date'] == cur].shape[0]
        assert(num1 < 60)
    except:
        count += 1
        body += str(count) + '. There are in total ' + str(num1) + ' orders with more than one insertion in same order.<div>'
    try:
        num2 = list(a[a['date'] == cur].groupby(['accCode'])['date'].size()[a[a['date'] == cur].groupby(['accCode'])['date'].size() > 15].index)
        assert(len(num2) == 0)
    except:
        count += 1
        num2 = ', '.join([str(x) for x in num2])
        body += str(count) + '. ' + num2 + ' has more than 15 orders with more than one insertion in same order.<div>'        
    
    orderLog = orderLog[~(orderLog['order'].isin(a['order'].unique()))]

orderLog['isMsg'] = np.where(orderLog['updateType'] == 0, 
                             np.where(orderLog['mse'] == 100, 1, 0), np.nan)
orderLog['isMsg'] = orderLog.groupby(['order'])['isMsg'].ffill()


### Assertion 3:  check IPO stocks selling status
print('=======================================================================================')
print('3. IPO stocks selling (ars = 301, 302)')
if orderLog[(orderLog['ars'].isin([301, 302])) & (orderLog['date'] == cur)].shape[0] != 0:
    kk = orderLog[(orderLog['ars'].isin([301, 302])) & (orderLog['date'] == cur)]
    print(kk)
    try:
        assert(kk[kk['orderDirection1'] == 1].shape[0] == 0)
        print('we only sell, never buy')
    except:
        print('There are IPO buy side orders!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        count += 1
        num1 = kk[kk['orderDirection1'] == 1].shape[0]
        body += str(count) + '. There are ' + str(num1) + ' IPO buy side orders.<div>'
        print(kk[kk['orderDirection1'] == 1])
    kk1 = kk[kk['updateType'] == 0]
    kk1 = kk1.sort_values(by=['accCode', 'secid','clockAtArrival'])
    kk1['diff'] = kk1.groupby(['accCode', 'secid'])['clockAtArrival'].apply(lambda x: x-x.shift(1))
    kk1['diff'] = kk1['diff'].fillna(0)
    try:
        assert(kk1[kk1['diff'] < 10e6].shape[0] == 0)
        print('for each stock in the same account, there is no insertion within 10 seconds of the previous insertion')
    except:
        count += 1
        kk1 = kk1.reset_index()
        num2 = kk1[kk1['diff'] < 10e6].shape[0]
        body += str(count) + '. There are ' + str(num2) + ' over ' + str(kk1.shape[0]) + ' orders with insertion within 10 seconds for orders under same account same stock.<div>'
        print('There are insertion within 10 seconds for orders under same account same stock!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')        
        print(kk1[kk1['diff'] < 10e6])
    kk2 = kk[(kk['updateType'] == 1)]
    
    try:
        assert(kk2[kk2['duration'] < 3e6].shape[0] == 0)
        print('for each stock in the same account, the cancellation of an order happens more than 3 seconds after the insertion')
    except:
        count += 1
        num2 = kk2[kk2['duration'] < 3e6].shape[0]
        body += str(count) + '. There are ' + str(num2) + ' over ' + str(kk1.shape[0]) + ' orders with cancellation within 3 seconds after insertion.<div>'        
        print('There are cancellation within 3 seconds for orders under same account same stock!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(kk2[kk2['duration'] < 3e6])


### Assertion 4: check updateType == 7 orders, make sure updateType == 7 orders < 20 per account, < 100 in total
print('=======================================================================================')
print('4. updateType 7 orders')
if orderLog[(orderLog['updateType'] == 7) & (orderLog['date'] == cur)].shape[0] != 0:
    try:
        assert(orderLog[(orderLog['updateType'] == 7) & (orderLog['date'] == cur)].groupby(['date', 'accCode'])['order'].nunique().max() < 20)
    except:
        print('There are more than 20 updateType 7 orders per account')
        count += 1
        a = list(orderLog[(orderLog['updateType'] == 7) & (orderLog['date'] == cur)].groupby(['accCode'])['order'].nunique()[
            orderLog[(orderLog['updateType'] == 7) & (orderLog['date'] == cur)].groupby(['accCode'])['order'].nunique() >= 20
        ].index)
        a = ', '.join([str(x) for x in a])
        body += str(count) + '. ' + a + ' has more than 20 updateType 7 orders.<div>'
    try:      
        assert(orderLog[(orderLog['updateType'] == 7) & (orderLog['date'] == cur)]['order'].nunique() < 100)
    except:
        print('Ther are more than 100 updateType 7 orders in total')
        count += 1
        body += str(count) + '. There are more than 100 updateType 7 orders in total.<div>'
    

### Assertion 5: check updateType == 6 orders, make sure updateType == 6 orders < 5% per account
print('=======================================================================================')
print('5. updateType 6 orders')
k1 = orderLog[(orderLog['updateType'] == 6) & (orderLog['date'] == cur)].groupby(['accCode'])['order'].nunique().reset_index()
k2 = orderLog[(orderLog['date'] == cur)].groupby(['accCode'])['order'].nunique().reset_index()
k = pd.merge(k1, k2, on=['accCode'], how='left')
k['prob'] = k['order_x']/k['order_y']
try:
    assert(sum(k['prob'] >= 0.05) == 0)
except:
    print('There are accounts with more than 5% updateType 6 orders!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    print(k[k['prob'] >= 0.05])
    a = k[k['prob'] >= 0.05]['accCode'].unique()
    a = ', '.join([str(x) for x in a])
    count += 1
    body += str(count) + '. ' + a + ' has more than 5% updateType 6 orders.<div>'

### Assertion 6: check CYB orders, make sure all CYB stocks have absOrderSize < 30w
print('=======================================================================================')
print('6. CYB stocks order size < 30w')
try:
    cyb = orderLog[(orderLog['secid'] >= 2300000) & (orderLog['updateType'] == 0) & (orderLog['date'] == cur)]
    assert(cyb[cyb['absOrderSize'] > 300000].shape[0] == 0)
except:
    print('CYB stocks total absOrderSize >= 30w!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    num1 = cyb[cyb['absOrderSize'] > 300000].shape[0]
    count += 1
    body += str(count) + '. There are ' + str(num1) + ' orders with CYB absOrderSize > 30w.<div>'
    
     
### Assertion 7:  make sure there is no unexpected updateType 
print('=======================================================================================')
print('7. unexpected updateType')
def getTuple(x):
    return tuple(i for i in x)

checkLog = orderLog[~((orderLog['updateType'] == 4) & (orderLog.groupby(['order'])['updateType'].shift(-1) == 4))]
checkLog = checkLog.groupby(['order'])['updateType'].apply(lambda x: getTuple(x)).reset_index()
checkLog['status'] = np.where(checkLog['updateType'].isin([(0, 2, 4), (0, 2, 1, 4), (0, 2, 1, 2, 4), (0, 2, 4, 1, 4), (0, 4), (0, 1, 4), (0, 4, 1, 4), (0, 2, 2, 4), (0, 4, 2, 4), (0, 2, 2, 1, 4), (0, 2, 2, 4, 1, 4)]),0,
                     np.where(checkLog['updateType'].isin([(0, 2, 4, 1, 3), (0, 2, 4, 1, 4, 3), (0, 2, 1, 4, 3), (0, 4, 1, 3), (0, 1, 4, 3),
                                                               (0, 2, 2, 4, 1, 3), (0, 2, 2, 4, 1, 4, 3), (0, 2, 2, 1, 4, 3), (0, 4, 2, 4, 1, 3),
                                                               (0, 4, 2, 1, 3), (0, 4, 1, 4, 3), (0, 4, 1)]), 1,
                     np.where(checkLog['updateType'].isin([(0, 2, 1, 3), (0, 2, 2, 1, 3), (0, 2, 3), (0, 3), (0, 1, 3), (0, ), (0, 2), (0, 2, 1), (0, 2, 2)]), 2, 3)))

orderLog = pd.merge(orderLog, checkLog[['order', 'status']], how='left', on=['order'], validate='many_to_one')
orderLog = orderLog[orderLog['status'].isin([0, 1, 2])].reset_index(drop=True)

### Assertion 8:  make sure status==0 got all traded
print('=======================================================================================')
print('8. status == 0: all traded')
a = orderLog[(orderLog['status'] == 0)]
a = a.groupby(['date', 'order'])[['absOrderSizeCumFilled', 'absOrderSize']].max().reset_index()
a.columns = ['date', 'order', 'filled', 'total']
print('in total trade, any fill != total cases')
display(a[a['filled'] != a['total']])
if a[a['filled'] != a['total']].shape[0] > 0:
    removeOrderLs = a[a['filled'] != a['total']]['order'].unique()
    count += 1
    body += str(count) + '. There are ' + str(a[(a['filled'] != a['total']) & (a['date'] == cur)]['order'].nunique()) + \
    ' over ' + str(a[(a['date'] == cur)]['order'].nunique()) + ' orders status == 0 but not all traded.<div>'
    orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]
    
### Assertion 9:  make sure status==1 got partial traded
print('=======================================================================================')
print('9. status == 1: partial traded')
a = orderLog[orderLog['status'] == 1]
a = a.groupby(['order'])[['absOrderSizeCumFilled', 'absOrderSize']].max().reset_index()
a.columns = ['order', 'filled', 'total']
print('in partial trade, any fill >= total or fill is 0 cases for updateType 4')
display(a[(a['filled'] >= a['total']) | (a['filled'] == 0)])
if a[(a['filled'] >= a['total']) | (a['filled'] == 0)].shape[0] > 0:
    removeOrderLs = a[(a['filled'] >= a['total']) | (a['filled'] == 0)]['order'].unique()
    count += 1
    body += str(count) + '. There are ' + str(a[((a['filled'] >= a['total']) | (a['filled'] == 0)) & (a['date'] == cur)]['order'].nunique()) + \
    ' over ' + str(a[(a['date'] == cur)]['order'].nunique()) + ' orders status == 1 but not partial traded.<div>'
    orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]
    
### Assertion 10: make sure no cancellation within 1 sec
print('=======================================================================================')
print('10. no cancellation within 1 sec')
a = orderLog[(orderLog['updateType'] == 1) & (orderLog['duration'] < 1e6)]
print('any cancellation within 1 sec')
display(a)
if a[a['date'] == cur].shape[0] > 0:
    count += 1
    body += str(count) + '. There are ' + str(a[(a['date'] == cur)]['order'].nunique()) + ' orders cancel within 1s.<div>'    
if a.shape[0] > 0:
    removeOrderLs = a['order'].unique()
    orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]


### Assertion 11: make sure no order has shares > 80w or notional > 800w
print('=======================================================================================')
print('11. Orders with size > 80w or notional > 800w')
orderLog['orderNtl'] = orderLog['absOrderSize'] * orderLog['orderPrice']
if orderLog[(orderLog['absOrderSize'] > 800000) & (orderLog['date'] == cur)].shape[0] > 0:
    count += 1
    body += str(count) + '. There are ' + str(orderLog[(orderLog['absOrderSize'] > 800000) & (orderLog['date'] == cur)]['order'].nunique()) + ' orders shares > 80w.<div>'    
if orderLog[orderLog['absOrderSize'] > 800000].shape[0] > 0:
    print('some order quantity are > 80w')
    print(orderLog[orderLog['absOrderSize'] > 800000].groupby(['colo', 'accCode'])['order'].nunique())
    display(orderLog[orderLog['absOrderSize'] > 800000][['date', 'accCode', 'secid', 'vai', 'absOrderSize', 'orderPrice',
                                                         'orderNtl', 'orderDirection', 'clock', 'order']])
if orderLog[(orderLog['orderNtl'] > 8000000) & (orderLog['date'] == cur)].shape[0] > 0:
    count += 1
    body += str(count) + '. There are ' + str(orderLog[(orderLog['orderNtl'] > 8000000) & (orderLog['date'] == cur)]['order'].nunique()) + ' orders notional > 800w.<div>'                
if orderLog[orderLog['orderNtl'] > 8000000].shape[0] > 0:
    print('some order ntl are > 800w')
    print(orderLog[orderLog['orderNtl'] > 8000000].groupby(['colo', 'accCode'])['order'].nunique())
    display(orderLog[orderLog['orderNtl'] > 8000000][['date', 'accCode', 'secid', 'vai', 'absOrderSize', 'orderPrice',
                                                      'orderNtl', 'orderDirection', 'clock', 'order', "updateType", 
                                                      "tradePrice", "absOrderSizeCumFilled", "absFilledThisUpdate"]])

removeOrderLs = list(set(orderLog[orderLog['absOrderSize'] > 800000]['order'].unique()) | set(orderLog[orderLog['orderNtl'] > 8000000]['order'].unique()))
orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]


orderLog = orderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)

orderLog['exchange'] = np.where(orderLog['secid'] >= 2000000, 'SZE', 'SSE')
orderLog['orderNtl'] = orderLog['orderPrice'] * orderLog['absOrderSize']
orderLog['tradeNtl'] = np.where(orderLog['updateType'] == 4, orderLog['tradePrice']*orderLog['absFilledThisUpdate'], 0)
orderLog["mrstaat"] = orderLog.groupby(['order'])['mrstaat'].transform('first')
orderLog["ars"] = orderLog.groupby(['order'])['ars'].transform('first')
orderLog["mrstauc"] = orderLog.groupby(['order'])['mrstauc'].transform('first')
orderLog["mrsb90"] = orderLog.groupby(['order'])['mrsb90'].transform('first')
orderLog["mrss90"] = orderLog.groupby(['order'])['mrss90'].transform('first')
orderLog["aaa"] = orderLog.groupby(['order'])['aaa'].transform('first')
orderLog = orderLog[~orderLog['ars'].isnull()]
# orderLog = orderLog[orderLog['ars'] % 10 == 1]


orderLog['m1'] = orderLog['mrstaat'].apply(lambda x: x - (x // 10000) * 10000)
orderLog['m2'] = orderLog['mrstauc'].apply(lambda x: x - (x // 10000) * 10000)
if orderLog[orderLog['mrsb90'] == '-'].shape[0] != 0:
    display(orderLog[orderLog['mrsb90'] == '-'])
orderLog = orderLog[orderLog['mrsb90'] != '-']
orderLog['mrsb90'] = orderLog['mrsb90'].astype(float)
if orderLog[orderLog['aaa'] == '-'].shape[0] != 0:
    display(orderLog[orderLog['aaa'] == '-'])
orderLog = orderLog[orderLog['aaa'] != '-']
orderLog['aaa'] = orderLog['aaa'].astype(float)
orderLog.loc[(orderLog['orderDirection'] >= 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'mrstauc'] = \
orderLog.loc[(orderLog['orderDirection'] >= 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'm2']

orderLog.loc[(orderLog['orderDirection'] >= 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'mrstaat'] = \
orderLog.loc[(orderLog['orderDirection'] >= 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'm1']

orderLog.loc[(orderLog['orderDirection'] < 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'mrstauc'] = \
orderLog.loc[(orderLog['orderDirection'] < 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'm2']

orderLog.loc[(orderLog['orderDirection'] < 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'mrstaat'] = \
orderLog.loc[(orderLog['orderDirection'] < 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'm1']    


orderLog['sta'] = np.where(orderLog['mrstaat'] == 1000, '1. staone', np.where(
orderLog['mrstaat'] == 3000, '2. statwo', np.where(
orderLog['mrstaat'].isin([11000, 13000]), '3. sta300', 'else')))
display(orderLog[(orderLog['sta'] == 'else') & (orderLog['updateType'] == 0)].groupby(['date', 'accCode'])['secid'].size())
display(orderLog[(orderLog['sta'] == 'else') & (orderLog['date'] == cur) & (orderLog['updateType'] == 0)])
m1 = orderLog[(orderLog['sta'] == 'else') & (orderLog['date'] == cur) & (orderLog['updateType'] == 0)].shape[0]
if m1 != 0:
    count += 1
    body += str(count) + '. There are ' + str(m1) + ' orders with invalid strategy.<div>'                    
orderLog = orderLog[orderLog['mrstaat'].isin([11000, 13000, 1000, 3000])]
display(orderLog[orderLog['updateType'] == 0].groupby(['mrstaat', 'mrstauc'])['date'].size())

-----------------------------------------------------------------------------------------------
load data


C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (1,54,62) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (54,62) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


'There are accounts with duplicated ticks:'

date      colo      accCode
20201130  zs_92_02  9243       2150
Name: ars, dtype: int64

'There are ticks with orderDirection 0'

...

11. Orders with size > 80w or notional > 800w


C:\Users\win\Anaconda3\lib\site-packages\pandas\core\ops\__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


date      accCode
20201123  5225       1
          5287       1
          5470       1
          5474       1
          6623       1
          6627       2
          6683       1
          8854       1
          8886       1
          9454       1
          9461       1
          9471       1
          9561       1
          9685       2
          522201     1
          522501     1
          522601     1
          526901     1
          527301     2
          527501     1
          527601     1
          527701     1
          528101     1
          528401     1
          528701     1
          528901     1
          529001     1
          529101     1
          966701     1
          968501     2
          974101     1
20201124  6237       1
          8854       1
          8886       1
          9685       1
          522401     1
          528401     1
20201125  5386       1
          9765       1
          968501     1
20201126  5386       1
20201127  6237       1
          9685  

,ApplSeqNum,aaa,absFilledThisUpdate,absOrderSize,absOrderSizeCumFilled,accCode,ars,caamd,cancellationPending,cfe,clock,clockAtArrival,cumSharesBought,cumSharesBuyInserted,cumSharesSellInserted,cumSharesSold,date,finalState,gfe,hee,insertedShortOrder,insertionPending,internalId,inv_L,inv_L0,inv_S,inv_S0,l4tr,locateShares,locateSharesTotal,mfe,mra100,mrb100,mrm,mrm25,mrmum,mrrlma,mrsb300,mrsb90,mrss300,mrss90,mrstaat,mrstauc,mrstaum,mrv,ms,mse,mt,mta,mv,orderDirection,orderId,orderOutstanding,orderPrice,orderSysId,sdd,secid,sequenceNo,session,threadId,totalActions,totalCanceled,tradeId,tradePrice,underlyingIndex,updateType,vai,zipFile,colo,test,broker,colo_broker,order,group,startClock,duration,orderDirection1,directNum,isMsg,status,orderNtl,exchange,tradeNtl,m1,m2,sta
8319557,0.0,0.000000,0,735,0,8854,0.0,0,0.0,2.667300e+06,2020-12-07 13:41:46.578785,1607319706578785,0.0,0.0,735.0,0.0,20201207,0.0,2.667299e+06,0.000000,0.0,1.0,2038.0,735.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,13:41:46.578130,23,0.0,4.618934e+07,2.310125e+07,-1,-1.0,0.0,17.23,NaN,134146000.0,1601686,90141116.0,0,57478.0,2274.0,236.0,NaN,-1.0,852,0,302.0,logs_20201207_zt_88_06_day_8854,zt_88_06,2144520,88,zt_88,2146853,1115987,1607319706578785,0,-1,1,0.0,0,12664.05,SSE,0.0,0.0,0.0,else
8319560,0.0,0.000000,0,735,0,8886,0.0,0,0.0,1.326839e+07,2020-12-07 13:41:46.578787,1607319706578787,0.0,0.0,735.0,0.0,20201207,0.0,1.326839e+07,0.000000,0.0,1.0,452.0,735.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,13:41:46.578130,23,0.0,4.618934e+07,7.424000e+07,-1,-1.0,0.0,17.23,NaN,134146000.0,1601686,90141116.0,0,57108.0,594.0,142.0,NaN,-1.0,905,0,302.0,logs_20201207_zt_88_06_day_8886,zt_88_06,2148524,88,zt_88,2150856,1115987,1607319706578787,0,-1,1,0.0,0,12664.05,SSE,0.0,0.0,0.0,else
8319563,0.0,0.000000,0,735,0,9451,0.0,0,0.0,4.197233e+06,2020-12-07 13:41:47.403411,1607319707403411,0.0,0.0,735.0,0.0,20201207,0.0,4.197232e+06,0.000000,0.0,1.0,605.0,735.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,13:41:47.403032,4,0.0,4.618934e+07,5.480964e+06,-1,-1.0,0.0,17.23,NaN,134146000.0,1601686,161648393.0,0,12684.0,605.0,37.0,NaN,-1.0,852,0,302.0,logs_20201207_zs_94_05_day_9451,zs_94_05,2086314,94,zs_94,2088653,1115987,1607319707403411,0,-1,1,0.0,0,12664.05,SSE,0.0,0.0,0.0,else
8319566,0.0,0.000000,0,707,0,9461,0.0,0,0.0,6.588006e+05,2020-12-07 13:41:44.787862,1607319704787862,0.0,0.0,707.0,0.0,20201207,0.0,6.587996e+05,0.000000,0.0,1.0,685.0,707.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,13:41:44.786511,4,0.0,4.618934e+07,1.353499e+07,-1,-1.0,0.0,17.23,NaN,134146000.0,1601686,161652955.0,0,29189.0,685.0,84.0,NaN,-1.0,300,0,302.0,logs_20201207_zs_94_03_day_9461,zs_94_03,2085688,94,zs_94,2088027,1115987,1607319704787862,0,-1,1,0.0,0,12181.61,SSE,0.0,0.0,0.0,else
8319571,0.0,0.000000,0,735,0,9685,0.0,0,0.0,3.696825e+06,2020-12-07 13:41:46.573063,1607319706573063,0.0,0.0,735.0,0.0,20201207,0.0,3.696824e+06,0.000000,0.0,1.0,1175.0,735.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,13:41:46.572824,13,0.0,4.618934e+07,8.822083e+07,-1,-1.0,0.0,17.23,NaN,134146000.0,1601686,160634963.0,0,6233.0,1367.0,192.0,NaN,-1.0,852,0,302.0,logs_20201207_zs_96_08_day_9685,zs_96_08,2105557,96,zs_96,2107890,1115987,1607319706573063,0,-1,1,0.0,2,12664.05,SSE,0.0,0.0,0.0,else
8319572,0.0,0.000000,0,735,0,522401,0.0,0,0.0,1.556404e+07,2020-12-07 13:41:46.528076,1607319706528076,0.0,0.0,735.0,0.0,20201207,0.0,1.556404e+07,0.000000,0.0,1.0,246471.0,735.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,13:41:46.527653,23,0.0,4.618934e+07,7.646108e+07,-1,-1.0,0.0,17.23,NaN,134146000.0,1601686,117767964.0,0

mrstaat  mrstauc
1000.0   0.0         315145
3000.0   1000.0     1482610
         2000.0      107244
         3000.0       45764
11000.0  10000.0       9371
         20000.0        870
         30000.0        652
13000.0  11000.0     175136
         12000.0       7201
         13000.0       2128
         21000.0       4875
         22000.0       5595
         23000.0       1025
         31000.0       2470
         32000.0       2080
         33000.0       2649
Name: date, dtype: int64

In [84]:
test = rawOrderLog[rawOrderLog['updateType'] == 0]
test['orderDirection1'] = np.where(test['orderDirection'].isin([1, 2]), 1, -1)
test['num1'] = test.groupby(['date', 'secid', 'vai'])['orderDirection1'].transform('nunique')
display(test[test['num1'] == 2].groupby(['ars'])['date'].size())
test[test['num1'] == 2].sort_values(by=['date', 'secid', 'vai', 'ars'])[['date', 'colo', 'accCode', 'secid', 'vai', 'ars', 'orderPrice', 'orderDirection', 'absOrderSize', 'absFilledThisUpdate', 'internalId', 'sdd', 'mra100', 'mrb100']].head(50)

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


ars
1.0    226
2.0    680
3.0      4
Name: date, dtype: int64

,date,colo,accCode,secid,vai,ars,orderPrice,orderDirection,absOrderSize,absFilledThisUpdate,internalId,sdd,mra100,mrb100
18570,20201119,zs_96_08,6237,1600132,644802,2.0,102.47,-1,600,0,1.605746e+15,100014000.0,10248.0,10216.0
18574,20201119,zs_96_08,6282,1600132,644802,2.0,102.17,1,600,0,1.605746e+15,100014000.0,10248.0,10216.0
18925,20201119,zs_96_08,6237,1600132,1878402,2.0,103.05,1,600,0,1.605746e+15,110259000.0,10343.0,10304.0
18933,20201119,zs_96_08,6282,1600132,1878402,2.0,103.05,1,100,0,1.605746e+15,110259000.0,10343.0,10304.0
18937,20201119,zt_94_06,9441,1600132,1878402,2.0,103.42,-1,600,0,8.390000e+02,110259000.0,10343.0,10304.0
18988,20201119,zs_96_08,6237,1600132,1954302,2.0,102.67,1,500,0,1.605746e+15,111130000.0,10300.0,10266.0
18992,20201119,zt_94_06,9441,1600132,1954302,2.0,102.99,-1,600,0,9.340000e+02,111130000.0,10300.0,10266.0
22989,20201119,zs_96_08,6237,1600143,20662497,2.0,16.43,-1,6900,0,1.605746e+15,110234000.0,1644.0,1636.0
22995,20201119,zs_96_08,6282,1600143,20662497,2.0,16.37,1,200,0,1.605746e+15,110234000.0,1644.0,1636.0
22999,20201119,zt_94_06,9441,1600143,20662497,2.0,16.43,-1,6900,0,8.330000e+02,110234000.0,1644.0,1636.0


In [43]:
rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['vai'] == 252100) & (rawOrderLog['accCode'] == 5456) & (rawOrderLog['secid'] == 2000910)][['date', 'colo', 'accCode', 'secid', 'vai', 'ars', 'orderPrice', 'orderDirection', 'absOrderSize', 'absFilledThisUpdate', 'internalId', 'sdd', 'mra100', 'mrb100']]

,date,colo,accCode,secid,vai,ars,orderPrice,orderDirection,absOrderSize,absFilledThisUpdate,internalId,sdd,mra100,mrb100
7475448,20201201,zs_54_01,5456,2000910,252100,2.0,16.08,1,900,0,71.0,93414960.0,1616.0,1608.0
7475449,20201201,zs_54_01,5456,2000910,252100,3.0,16.10,1,900,0,77.0,93417570.0,1615.0,1610.0


In [77]:
test[(test['ars'] == 2) & (test['orderDirection'].isin([-1,-2])) & (test['accCode'] == 5456) & ((test['orderPrice'] <= test['mrb100']/100) | (test['orderPrice'] >= test['mra100']/100))][['date', 'colo', 'accCode', 'secid', 'vai', 'ars', 'orderPrice', 'orderDirection', 'absOrderSize', 'absFilledThisUpdate', 'internalId', 'sdd', 'mra100', 'mrb100']]

,date,colo,accCode,secid,vai,ars,orderPrice,orderDirection,absOrderSize,absFilledThisUpdate,internalId,sdd,mra100,mrb100
7475713,20201201,zs_54_01,5456,2000913,162700,2.0,25.76,-1,1100,0,104.0,93447500.0,2576.0,2565.0
7475714,20201201,zs_54_01,5456,2000913,170700,2.0,25.87,-1,1100,0,120.0,93502140.0,2587.0,2578.0
7475716,20201201,zs_54_01,5456,2000913,204100,2.0,25.75,-1,1100,0,177.0,93605700.0,2575.0,2564.0
7488911,20201201,zs_54_01,5456,2002034,232940,2.0,16.71,-1,600,0,561.0,95727010.0,1671.0,1665.0
7488912,20201201,zs_54_01,5456,2002034,242240,2.0,16.74,-1,600,0,572.0,95912650.0,1674.0,1669.0
7488916,20201201,zs_54_01,5456,2002034,375940,2.0,16.80,-1,600,0,670.0,101636500.0,1680.0,1671.0
7488917,20201201,zs_54_01,5456,2002034,379340,2.0,16.79,-1,600,0,672.0,101642650.0,1679.0,1675.0
7488920,20201201,zs_54_01,5456,2002034,390040,2.0,16.79,-1,600,0,676.0,101748930.0,1679.0,1672.0
7488921,20201201,zs_54_01,5456,2002034,390040,2.0,16.79,-1,600,0,678.0,101754930.0,1679.0,1673.0
7550920,20201201,zs_54_01,5456,2002389,1021700,2.0,25.67,-1,4600,0,1.0,93315950.0,2567.0,2557.0


In [66]:
test[(test['ars'] == 2) & ((test['orderPrice'] <= test['mrb100']/100) | (test['orderPrice'] >= test['mra100']/100))].groupby(['date', 'accCode'])['secid'].size()

date      accCode
20201123  8854          1
20201201  5456        102
          6237       2089
          6282       1864
          9441        240
          9461        159
          9758        889
20201202  5456        124
          6237       1779
          6282       1629
          9441        425
          9461         88
          9758        981
20201203  6237       1764
          6282       1590
Name: secid, dtype: int64

In [125]:
r1 = test[(test['ars'] == 2) & (test['orderPrice'] == test['mra100']/100) & (test['orderDirection'].isin([-1, -2]))].groupby(['date', 'accCode'])['secid'].size().reset_index()
r2 = test[(test['ars'] == 2) & (test['orderDirection'].isin([-1, -2]))].groupby(['date', 'accCode'])['secid'].size().reset_index()
r1 = r1.rename(columns={'secid':'num1'})
r2 = r2.rename(columns={'secid':'num2'})
re = pd.merge(r1, r2, on=['date', 'accCode'], how='outer')
re['num1'] = re['num1'].fillna(0).astype(int)
HTML(re[re['date'] >= 20201201].sort_values(by=['date', 'accCode']).groupby(['accCode', 'date']).first().to_html())

In [128]:
rawOrderLog[(rawOrderLog['ars'] == 2) & (rawOrderLog['date'] < 20201201) & (rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] == 6237)][['date', 'updateType', 'colo', 'accCode', 'secid', 'vai', 'ars', 'orderPrice', 'orderDirection', 'absOrderSize', 'absFilledThisUpdate', 'internalId', 'sdd', 'mra100', 'mrb100']]

,date,updateType,colo,accCode,secid,vai,ars,orderPrice,orderDirection,absOrderSize,absFilledThisUpdate,internalId,sdd,mra100,mrb100
6086,20201119,0,zs_96_08,6237,1600053,1156908,2.0,23.51,1,400,0,1.605746e+15,144656000.0,2362.0,2350.0
6098,20201119,0,zs_96_08,6237,1600053,1172708,2.0,23.51,1,400,0,1.605746e+15,144720000.0,2360.0,2350.0
18562,20201119,0,zs_96_08,6237,1600132,629702,2.0,102.39,-1,600,0,1.605746e+15,95934000.0,10240.0,10228.0
18570,20201119,0,zs_96_08,6237,1600132,644802,2.0,102.47,-1,600,0,1.605746e+15,100014000.0,10248.0,10216.0
18578,20201119,0,zs_96_08,6237,1600132,647502,2.0,102.47,-1,600,0,1.605746e+15,100023000.0,10248.0,10222.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7052546,20201130,0,zs_96_08,6237,2300630,1491462,2.0,43.16,1,900,0,1.606696e+15,100415620.0,4330.0,4315.0
7052550,20201130,0,zs_96_08,6237,2300630,3223553,2.0,42.92,1,900,0,1.606696e+15,110325520.0,4305.0,4291.0
7052551,20201130,0,zs_96_08,6237,2300630,3307453,2.0,42.85,1,900,0,1.606696e+15,110738880.0,4297.0,4284.0
7052553,20201130,0,zs_96_08,6237,2300630,3314353,2.0,42.84,1,900,0,1.606696e+15,110816130.0,4299.0,4283.0


In [112]:
test[(test['ars'] == 3) & (test['date'] == 20201119)][['date', 'updateType', 'colo', 'accCode', 'secid', 'vai', 'ars', 'orderPrice', 'orderDirection', 'absOrderSize', 'absFilledThisUpdate', 'internalId', 'sdd', 'mra100', 'mrb100']]

,date,updateType,colo,accCode,secid,vai,ars,orderPrice,orderDirection,absOrderSize,absFilledThisUpdate,internalId,sdd,mra100,mrb100
1650,20201119,0,zt_88_06,8854,1600020,5028799,3.0,3.53,-1,200,0,2705.0,144538000.0,353.0,352.0
4862,20201119,0,zt_96_01,966301,1600037,3963001,3.0,10.60,1,1800,0,3.0,130134000.0,1061.0,1060.0
4866,20201119,0,zt_96_01,966301,1600037,4010501,3.0,10.60,1,1800,0,66.0,130414000.0,1061.0,1060.0
4870,20201119,0,zt_96_01,966301,1600037,4061001,3.0,10.60,1,1800,0,84.0,130637000.0,1061.0,1060.0
4875,20201119,0,zt_96_01,966301,1600037,4118901,3.0,10.59,1,800,0,91.0,130843000.0,1060.0,1059.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
705261,20201119,0,zt_88_06,8854,2300366,8147919,3.0,11.38,1,3100,0,1955.0,140141260.0,1139.0,1138.0
705300,20201119,0,zt_88_06,8854,2300366,8304919,3.0,11.34,1,600,0,2116.0,140658260.0,1135.0,1134.0
705304,20201119,0,zt_88_06,8854,2300366,8463381,3.0,11.34,1,600,0,2223.0,141243070.0,1135.0,1134.0
748805,20201119,0,zt_88_06,8854,2300527,15678413,3.0,11.09,-1,5100,0,167.0,130321640.0,1109.0,1108.0


In [116]:
test[(test['ars'] == 2) & (test['orderPrice'] < test['mra100']/100) & (test['orderDirection'].isin([-1, -2])) & (test['date'] >= 20201201)][['date', 'updateType', 'colo', 'accCode', 'secid', 'vai', 'ars', 'orderPrice', 'orderDirection', 'absOrderSize', 'absFilledThisUpdate', 'internalId', 'sdd', 'mra100', 'mrb100']]

,date,updateType,colo,accCode,secid,vai,ars,orderPrice,orderDirection,absOrderSize,absFilledThisUpdate,internalId,sdd,mra100,mrb100
7181191,20201201,0,zt_88_06,8854,1600389,1979101,2.0,21.43,-1,900,0,858.0,140218000.0,2144.0,2141.0
7181199,20201201,0,zt_88_06,8854,1600389,1999368,2.0,21.41,-1,900,0,871.0,140439000.0,2142.0,2139.0
7181206,20201201,0,zt_88_06,8854,1600389,2063168,2.0,21.44,-1,900,0,904.0,141601000.0,2145.0,2142.0
7181207,20201201,0,zt_88_06,8854,1600389,2119568,2.0,21.49,-1,900,0,937.0,142355000.0,2150.0,2147.0
7185881,20201201,0,zt_96_01,966301,1600426,14672708,2.0,35.60,-1,100,0,114.0,141621000.0,3561.0,3550.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9129371,20201203,0,zt_88_06,8854,2300748,11563109,2.0,41.98,-1,300,0,434.0,133106000.0,4199.0,4195.0
9139070,20201203,0,zt_88_06,8854,2300777,3955917,2.0,47.50,-1,300,0,435.0,133109000.0,4751.0,4747.0
9143843,20201203,0,zt_88_06,8854,2300792,2330386,2.0,117.44,-1,300,0,145.0,130445000.0,11745.0,11738.0
9143844,20201203,0,zt_88_06,8854,2300792,2330986,2.0,117.43,-1,200,0,154.0,130451000.0,11744.0,11737.0


In [92]:
test[(test['ars'] == 2) & (test['orderDirection'].isin([1, 2]))].groupby(['date', 'accCode'])['secid'].size().reset_index()

,date,accCode,secid
0,20201119,5456,242
1,20201119,6237,739
2,20201119,6282,754
3,20201119,8854,262
4,20201119,9441,203
5,20201119,9461,95
6,20201119,9758,593
7,20201119,966301,65
8,20201120,5456,209
9,20201120,6237,568


In [79]:
rawOrderLog[(rawOrderLog['date'] == 20201201) & (rawOrderLog['secid'] == 2000913) & (rawOrderLog['internalId'] == 104) & (rawOrderLog['accCode'] == 5456)].sort_values(by='clockAtArrival')[['date', 'updateType', 'colo', 'accCode', 'secid', 'vai', 'ars', 'orderPrice', 'orderDirection', 'absOrderSize', 'absFilledThisUpdate', 'internalId', 'sdd', 'mra100', 'mrb100']]

,date,updateType,colo,accCode,secid,vai,ars,orderPrice,orderDirection,absOrderSize,absFilledThisUpdate,internalId,sdd,mra100,mrb100
7475713,20201201,0,zs_54_01,5456,2000913,162700,2.0,25.76,-1,1100,0,104.0,93447500.0,2576.0,2565.0
7475581,20201201,2,zs_54_01,5456,2000913,-1,-1.0,25.76,-1,1100,0,104.0,-1.0,2576.0,2565.0
7475582,20201201,4,zs_54_01,5456,2000913,-1,-1.0,25.76,-1,1100,1100,104.0,-1.0,2576.0,2569.0


In [19]:
rawOrderLog['ars'].fillna(0).astype(int).unique()

array([       1,       -1,        2,        4,        3,      102,
              0,      103,      101,      105,      104, 17546528,
        3070573,  2571370,  3277503,  2933021], dtype=int64)

### 1. fill rate

In [3]:
body += '<p><b>II. fill rate</b></p>'

import pandas as pd
def convertToHtml(result,title):
    #将数据转换为html的table
    #result是list[list1,list2]这样的结构
    #title是list结构；和result一一对应。titleList[0]对应resultList[0]这样的一条数据对应html表格中的一列
    d = {}
    index = 0
    for t in title:
        d[t]=result[index]
        index = index+1
    df = pd.DataFrame(d)
    df = df[title]
    h = df.to_html(index=False)
    return h

placeSZE = orderLog[(orderLog['secid'] >= 2000000) & (orderLog['updateType'] == 0)]
print('%.2f%% SZE orders triggered by msg data'%(placeSZE[placeSZE['isMsg'] == 1].shape[0]/placeSZE.shape[0]*100))
body += '%.2f%% SZE orders triggered by msg data<div>'%(placeSZE[placeSZE['isMsg'] == 1].shape[0]/placeSZE.shape[0]*100)
placeSZE = orderLog[(orderLog['secid'] >= 2000000) & (orderLog['updateType'] == 0)]
r1 = placeSZE[placeSZE['isMsg'] == 1].groupby(['date', 'accCode'])['secid'].size().reset_index()
r2 = placeSZE.groupby(['date', 'accCode'])['secid'].size().reset_index()
re = pd.merge(r1, r2, on=['date', 'accCode'], how='inner')
re['perc'] = re['secid_x'] / re['secid_y']
print(re[re['perc'] < 0.8])
for i in re[re['perc'] < 0.8]['accCode'].unique():
    a = list(re[(re['perc'] < 0.8) & (re['accCode'] == i)]['date'].unique())
    a = ', '.join([str(x) for x in a])
    body += 'accCode ' + str(i) + ' SZE orders on ' + a + ' has msg triggered percentage < 80%<div>'

orderLog['tag'] = 'previous'
orderLog.loc[orderLog['date'] == cur, 'tag'] = 'current'
o1 = orderLog[orderLog['updateType'] == 0].groupby(['tag', 'exchange'])['orderNtl'].sum().reset_index()
o2 = orderLog[orderLog['updateType'] == 4].groupby(['tag', 'exchange'])['tradeNtl'].sum().reset_index()
o = pd.merge(o1, o2, on=['tag', 'exchange'])
o['perc'] = o['tradeNtl'] / o['orderNtl']
o['perc'] = o['perc'].apply(lambda x: '%.f'%(x*100))
o1 = o[o['tag'] == 'current']
o1 = o1.rename(columns={'perc':'cur_perc'})
o2 = o[o['tag'] == 'previous']
o2 = o2.rename(columns={'perc':'prev_perc'})
re = pd.merge(o1[['exchange', 'cur_perc']], o2[['exchange', 'prev_perc']], on='exchange')
from IPython.display import display, HTML
display(HTML(re.groupby(['exchange']).first().to_html()))
result = [re['exchange'].values, re['cur_perc'].values, re['prev_perc'].values]
title=['exchange', 'cur_perc', 'prev_perc']
body += convertToHtml(result,title)

orderLog['num'] = orderLog.groupby(['exchange', 'colo', 'accCode', 'secid'])['tag'].transform('nunique')
checkLog = orderLog[orderLog['num'] == 2]
# checkLog = orderLog
d1 = checkLog[(checkLog['updateType'] == 0)].groupby(['tag', 'exchange', 'sta', 'colo', 'accCode'])['orderNtl'].sum().reset_index()
d2 = checkLog[(checkLog['updateType'] == 4)].groupby(['tag', 'exchange', 'sta', 'colo', 'accCode'])['tradeNtl'].sum().reset_index()
dd = pd.merge(d1, d2, on=['tag', 'exchange', 'sta', 'colo', 'accCode'])
dd['fill rate'] = dd['tradeNtl'] / dd['orderNtl']
add = checkLog[(checkLog['updateType'] == 0)].groupby(['tag', 'exchange', 'sta', 'colo', 'accCode'])['order'].nunique().reset_index()
add = add.rename(columns={'order':'num'})
dd = pd.merge(dd, add, on=['tag', 'exchange', 'sta', 'colo', 'accCode'])
prev = dd[dd['tag'] == 'previous']
prev = prev.rename(columns={'num':'prev_num', 'fill rate':'prev_fillRate'})
cur = dd[dd['tag'] == 'current']
cur = cur.rename(columns={'num':'cur_num', 'fill rate':'cur_fillRate'})
report = pd.merge(prev[['exchange', 'sta', 'colo', 'accCode', 'prev_num', 'prev_fillRate']], 
              cur[['exchange', 'sta', 'colo', 'accCode', 'cur_num', 'cur_fillRate']], on=['exchange', 'sta', 'colo', 'accCode'], how='inner')

a1 = checkLog[(checkLog['updateType'] == 0) & (checkLog['tag'] == 'previous')].groupby(['date', 'exchange', 'sta', 'colo', 'accCode'])['orderNtl'].sum().reset_index()
a2 = checkLog[(checkLog['updateType'] == 4) & (checkLog['tag'] == 'previous')].groupby(['date', 'exchange', 'sta', 'colo', 'accCode'])['tradeNtl'].sum().reset_index()
aa = pd.merge(a1, a2, on=['date', 'exchange', 'sta', 'colo', 'accCode'])
aa['fill rate'] = aa['tradeNtl'] / aa['orderNtl']
aa['count'] = aa.groupby(['exchange', 'sta', 'colo', 'accCode'])['date'].transform('nunique')
aa1 = aa.groupby(['exchange', 'sta', 'colo', 'accCode'])['fill rate'].describe()[['mean', 'std']].reset_index()
aa1 = aa1.fillna(0)
aa = pd.merge(aa, aa1, on=['exchange', 'sta', 'colo', 'accCode'])
for j in [1]:
    aa[str(j) + 'std_low'] = aa['mean'] - j*aa['std']
    aa[str(j) + 'std_high'] = aa['mean'] + j*aa['std']
    aa['count1'] = np.where((aa['fill rate'] <= aa[str(j) + 'std_high']) & (aa['fill rate'] >= aa[str(j) + 'std_low']), 1, 0)
    re1 = aa.groupby(['exchange', 'sta', 'colo', 'accCode'])['count1'].sum().reset_index()
    re2 = aa.groupby(['exchange', 'sta', 'colo', 'accCode'])['count'].first().reset_index()
    re1 = pd.merge(re2, re1, on=['exchange', 'sta', 'colo', 'accCode'])
    re1['count1'] = re1['count1'] / re1['count']
    re1 = re1.rename(columns={'count1':str(j)+'*std'})
for j in [1.5, 2, 2.5, 3, 3.5, 4, 4.5]:
    aa[str(j) + 'std_low'] = aa['mean'] - j*aa['std']
    aa[str(j) + 'std_high'] = aa['mean'] + j*aa['std']
    aa['count1'] = np.where((aa['fill rate'] <= aa[str(j) + 'std_high']) & (aa['fill rate'] >= aa[str(j) + 'std_low']), 1, 0)
    re = aa.groupby(['exchange', 'sta', 'colo', 'accCode'])['count1'].sum().reset_index()
    re2 = aa.groupby(['exchange', 'sta', 'colo', 'accCode'])['count'].first().reset_index()
    re = pd.merge(re2, re, on=['exchange', 'sta', 'colo', 'accCode'])
    re['count1'] = re['count1'] / re['count']
    re = re.rename(columns={'count1':str(j)+'*std'})
    re1 = pd.merge(re1, re[['exchange', 'sta', 'colo', 'accCode', str(j)+'*std']], on=['exchange', 'sta', 'colo', 'accCode'])
nc = []
for i in range(0, re1.shape[0]):
    nc.append(np.float(re1.columns[5:][(re1.iloc[i, 5:] == 1)][0].split('*')[0]))
re1['n'] = nc
display(re1.shape[0])
display(aa1.shape[0])
aa1 = pd.merge(aa1, re1[['exchange', 'sta', 'colo', 'accCode', 'n']], on=['exchange', 'sta', 'colo', 'accCode'])
aa1['min'] = aa1['mean'] - aa1['std'] * aa1['n']
aa1['max'] = aa1['mean'] + aa1['std'] * aa1['n']
report = pd.merge(report, aa1, on=['exchange', 'sta', 'colo', 'accCode'], how='left')
assert(report[report['mean'].isnull()].shape[0] == 0)
display(report[((report['cur_fillRate'] > report['max']) | (report['cur_fillRate'] < report['min'])) & (report['cur_num'] > 100) & (abs(report['cur_fillRate'] - report['prev_fillRate']) > 0.15)].groupby(['exchange', 'sta', 'colo', 'accCode'])['prev_fillRate', 'cur_fillRate'].first())
report = report[((report['cur_fillRate'] > report['max']) | (report['cur_fillRate'] < report['min'])) & (report['cur_num'] > 100) & (abs(report['cur_fillRate'] - report['prev_fillRate']) > 0.15)].groupby(['exchange', 'sta', 'colo', 'accCode'])['prev_fillRate', 'cur_fillRate'].first().reset_index()

for cols in ['prev_fillRate', 'cur_fillRate']:
    report[cols] = report[cols].apply(lambda x: '%.f%%'%(100*x))
from IPython.display import display, HTML
display(HTML(report.groupby(['exchange', 'sta', 'colo', 'accCode']).first().to_html()))
body += '<div>In the following cases, fill rate under given accCode pass the hurdle we set:'
result = [report['exchange'].values, report['sta'].values, report['colo'].values, report['accCode'].values, 
         report['prev_fillRate'].values, report['cur_fillRate'].values]
title = ['exchange', 'sta', 'colo', 'accCode', 'prev_fillRate', 'cur_fillRate']
body += convertToHtml(result,title)

99.28% SZE orders triggered by msg data
         date  accCode  secid_x  secid_y      perc
23   20201123     6480      459     2418  0.189826
279  20201127   966301       14       65  0.215385
337  20201130   966301        9      151  0.059603


,cur_perc,prev_perc
exchange,,
SSE,69,69
SZE,75,73


215

215

prev_fillRate  cur_fillRate
exchange sta       colo     accCode                             
SSE      2. statwo zs_96_08 974101        0.592960      0.812821
SZE      1. staone zs_88_04 8967          0.629237      0.898060
                   zs_96_08 9741          0.424513      0.824024
         2. statwo zs_96_08 9741          0.446191      0.772557

In [128]:
d1 = orderLog[(orderLog['tag'] == 'previous') & (orderLog['accCode'] == 5229) & (orderLog['updateType'] == 4) & (orderLog['sta'] == '2. statwo')].groupby('secid')['tradeNtl'].sum().reset_index()
d2 = orderLog[(orderLog['tag'] == 'previous') & (orderLog['accCode'] == 5229) & (orderLog['updateType'] == 0) & (orderLog['sta'] == '2. statwo')].groupby('secid')['orderNtl'].sum().reset_index()
d = pd.merge(d1, d2, on=['secid'])
d = d.fillna(0)
d['fillRate'] = (d['tradeNtl'] / d['orderNtl']) * 100
d['fillRate'].describe([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99]).astype('int')

count    226
mean      81
std       18
min        1
10%       58
20%       71
30%       76
40%       82
50%       85
60%       88
70%       94
80%       98
90%      100
95%      100
99%      100
max      100
Name: fillRate, dtype: int32

In [126]:
d1 = orderLog[(orderLog['tag'] == 'current') & (orderLog['accCode'] == 5229) & (orderLog['updateType'] == 4) & (orderLog['sta'] == '2. statwo')].groupby('secid')['tradeNtl'].sum().reset_index()
d2 = orderLog[(orderLog['tag'] == 'current') & (orderLog['accCode'] == 5229) & (orderLog['updateType'] == 0) & (orderLog['sta'] == '2. statwo')].groupby('secid')['orderNtl'].sum().reset_index()
dd = pd.merge(d1, d2, on=['secid'], how='outer')
dd = dd.fillna(0)
dd['fillRate'] = (dd['tradeNtl'] / dd['orderNtl']) * 100
dd['fillRate'].describe([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99]).astype('int')
re = pd.merge(d, dd, on='secid')
re[re['fillRate_y'] < re['fillRate_x']].sort_values(by='orderNtl_y', ascending=False)

,secid,tradeNtl_x,orderNtl_x,fillRate_x,tradeNtl_y,orderNtl_y,fillRate_y
94,2300595,3159887.00,4366538.00,72.365957,1580001.00,3938064.0,40.121263
88,2300357,5848823.61,7211036.66,81.109331,1290107.81,2922897.0,44.137984
93,2300496,13107774.74,14023521.10,93.469926,1637807.92,2432504.0,67.330122
33,2002156,14719929.65,15404061.06,95.558759,2049632.25,2402297.0,85.319686
83,2300285,5778486.00,6687566.00,86.406415,1422929.75,1761929.0,80.759767
44,2002353,13259323.66,16007975.00,82.829488,736002.00,1607593.0,45.782857
4,2000301,6320533.71,8820342.22,71.658600,637095.00,1606982.0,39.645435
13,2000738,4264176.00,5711564.00,74.658640,570377.00,1484957.0,38.410338
53,2002444,8633900.65,9770578.65,88.366319,571878.00,1235290.0,46.295040
60,2002595,2952160.00,3991028.00,73.969915,656506.00,1098557.0,59.760759


In [132]:
t = orderLog[(orderLog['secid'] == 2002004) & (orderLog['accCode'] == 5229) & (orderLog['tag'] == 'previous') & (orderLog['sta'] == '2. statwo')]
t[['date', 'colo', 'accCode', 'updateType', 'secid', 'vai', 'ars', 'ApplSeqNum', 'sdd', 'absOrderSize', 'orderPrice', 'absFilledThisUpdate', 'internalId', 'orderId']]

,date,colo,accCode,updateType,secid,vai,ars,ApplSeqNum,sdd,absOrderSize,orderPrice,absFilledThisUpdate,internalId,orderId
4108100,20201127,zs_52_08,5229,0,2002004,7380385.0,1.0,15964372.0,141439850.0,116000,5.44,0,92095.0,-1.0
4108101,20201127,zs_52_08,5229,2,2002004,7380385.0,1.0,0.0,-1.0,116000,5.44,0,92095.0,537509.0
4108102,20201127,zs_52_08,5229,4,2002004,7380385.0,1.0,0.0,-1.0,116000,5.44,2250,92095.0,537509.0
4108103,20201127,zs_52_08,5229,1,2002004,7380385.0,1.0,0.0,51281.0,116000,5.44,0,92095.0,537509.0
4108104,20201127,zs_52_08,5229,3,2002004,7380385.0,1.0,0.0,-1.0,116000,5.44,0,92095.0,537509.0


In [114]:
orderLog[(orderLog['accCode'] == 9741) & (orderLog['secid'] == 2002384) & (orderLog['updateType'] == 0)].groupby(['date', 'sta'])['order'].size()

date      sta      
20201125  1. staone     9
          2. statwo    22
20201126  2. statwo    10
20201127  2. statwo    21
20201201  1. staone     4
          2. statwo    39
20201202  1. staone     1
          2. statwo    16
20201203  2. statwo    20
20201204  1. staone     1
          2. statwo    17
20201207  1. staone     1
          2. statwo     5
Name: order, dtype: int64

In [133]:
t = orderLog[(orderLog['secid'] == 2002004) & (orderLog['accCode'] == 5229) & (orderLog['tag'] == 'current') & (orderLog['sta'] == '2. statwo')]
t[['date', 'colo', 'accCode', 'updateType', 'secid', 'vai', 'ars', 'ApplSeqNum', 'sdd', 'absOrderSize', 'orderPrice', 'absFilledThisUpdate', 'internalId', 'orderId']]

,date,colo,accCode,updateType,secid,vai,ars,ApplSeqNum,sdd,absOrderSize,orderPrice,absFilledThisUpdate,internalId,orderId


In [99]:
orderLog['ars'].unique()

array([1., 2., 3., 7.])

### 2. internal latency

In [4]:
body += '<p><b>III. internal latency</b></p>'

orderLog['num'] = orderLog.groupby(['exchange', 'colo', 'accCode', 'secid'])['tag'].transform('nunique')
checkLog = orderLog[(orderLog["updateType"] == 0) & (orderLog['num'] == 2)]
checkLog = checkLog[checkLog['caamd'] != 0]
checkLog['internal_latency'] = checkLog["clockAtArrival"] - checkLog["caamd"]
SZE = checkLog[checkLog['secid'] >= 2000000]
SSE = checkLog[checkLog['secid'] < 2000000]
SZE = SZE[SZE['isMsg'] == 1]
c1 = SZE.groupby(['tag', "exchange", "colo", "accCode", "sta", "isMsg"])["internal_latency"].quantile(.95).reset_index()
c2 = SZE.groupby(['tag', "exchange", "colo", "accCode", "sta", "isMsg"])["internal_latency"].median().reset_index()
c3 = SZE.groupby(['tag', "exchange", "colo", "accCode", "sta", "isMsg"])["internal_latency"].count().reset_index()

re1 = pd.merge(c3, c1, on=['tag', "exchange", "colo", "accCode", "sta", "isMsg"])
re1 = re1.rename(columns = {'internal_latency_x': 'count', 'internal_latency_y': '95 percentile'})
re1 = pd.merge(re1, c2, on=['tag', "exchange", "colo", "accCode", "sta", "isMsg"])
re1 = re1.rename(columns = {'internal_latency': 'median'})
re1['isMsg'] = 1

c1 = SSE.groupby(['tag', "exchange", "colo", "accCode", "sta", "isMsg"])["internal_latency"].quantile(.95).reset_index()
c2 = SSE.groupby(['tag', "exchange", "colo", "accCode", "sta", "isMsg"])["internal_latency"].median().reset_index()
c3 = SSE.groupby(['tag', "exchange", "colo", "accCode", "sta", "isMsg"])["internal_latency"].count().reset_index()

re2 = pd.merge(c3, c1, on=['tag', "exchange", "colo", "accCode", "sta", "isMsg"])
re2 = re2.rename(columns = {'internal_latency_x': 'count', 'internal_latency_y': '95 percentile'})
re2 = pd.merge(re2, c2, on=['tag', "exchange", "colo", "accCode", "sta", "isMsg"])
re2 = re2.rename(columns = {'internal_latency': 'median'})
re2

re = pd.concat([re1, re2]).reset_index(drop=True)

for col in ['isMsg','median', '95 percentile']:
    re[col] = re[col].astype(int)

re1 = re[re['tag'] == 'current']
re1 = re1.rename(columns={'count':'cur_count', 'median':'cur_med', '95 percentile':'cur_95p'})
re2 = re[re['tag'] == 'previous']
re2 = re2.rename(columns={'count':'prev_count', 'median':'prev_med', '95 percentile':'prev_95p'})
report = pd.merge(re1, re2, on=['exchange', 'colo', 'accCode', 'sta', 'isMsg'])

checkLog[(checkLog['exchange'] == 'SSE') | ((checkLog['exchange'] == 'SZE') & (checkLog['isMsg'] == 1))]
aa = checkLog[(checkLog['tag'] == 'previous')].groupby(['date', 'exchange', 'sta', 'colo', 'accCode', 'isMsg'])['internal_latency'].median().reset_index()
aa['count'] = aa.groupby(['exchange', 'sta', 'colo', 'accCode', 'isMsg'])['date'].transform('nunique')
aa1 = aa.groupby(['exchange', 'sta', 'colo', 'accCode', 'isMsg'])['internal_latency'].describe()[['mean', 'std']].reset_index()
aa1 = aa1.fillna(0)
aa = pd.merge(aa, aa1, on=['exchange', 'sta', 'colo', 'accCode', 'isMsg'])
for j in [1]:
    aa[str(j) + 'std_low'] = aa['mean'] - j*aa['std']
    aa[str(j) + 'std_high'] = aa['mean'] + j*aa['std']
    aa['count1'] = np.where((aa['internal_latency'] <= aa[str(j) + 'std_high']) & (aa['internal_latency'] >= aa[str(j) + 'std_low']), 1, 0)
    re1 = aa.groupby(['exchange', 'sta', 'colo', 'accCode', 'isMsg'])['count1'].sum().reset_index()
    re2 = aa.groupby(['exchange', 'sta', 'colo', 'accCode', 'isMsg'])['count'].first().reset_index()
    re1 = pd.merge(re2, re1, on=['exchange', 'sta', 'colo', 'accCode', 'isMsg'])
    re1['count1'] = re1['count1'] / re1['count']
    re1 = re1.rename(columns={'count1':str(j)+'*std'})
for j in [1.5, 2, 2.5, 3, 3.5, 4, 4.5]:
    aa[str(j) + 'std_low'] = aa['mean'] - j*aa['std']
    aa[str(j) + 'std_high'] = aa['mean'] + j*aa['std']
    aa['count1'] = np.where((aa['internal_latency'] <= aa[str(j) + 'std_high']) & (aa['internal_latency'] >= aa[str(j) + 'std_low']), 1, 0)
    re = aa.groupby(['exchange', 'sta', 'colo', 'accCode', 'isMsg'])['count1'].sum().reset_index()
    re2 = aa.groupby(['exchange', 'sta', 'colo', 'accCode', 'isMsg'])['count'].first().reset_index()
    re = pd.merge(re2, re, on=['exchange', 'sta', 'colo', 'accCode', 'isMsg'])
    re['count1'] = re['count1'] / re['count']
    re = re.rename(columns={'count1':str(j)+'*std'})
    re1 = pd.merge(re1, re[['exchange', 'sta', 'colo', 'accCode', 'isMsg', str(j)+'*std']], on=['exchange', 'sta', 'colo', 'accCode', 'isMsg'])
nc = []
for i in range(0, re1.shape[0]):
    nc.append(np.float(re1.columns[6:][(re1.iloc[i, 6:] == 1)][0].split('*')[0]))
re1['n'] = nc
display(re1.shape[0])
display(aa1.shape[0])
aa1 = pd.merge(aa1, re1[['exchange', 'sta', 'colo', 'accCode', 'isMsg', 'n']], on=['exchange', 'sta', 'colo', 'accCode', 'isMsg'])
aa1['min'] = aa1['mean'] - aa1['std'] * aa1['n']
aa1['max'] = aa1['mean'] + aa1['std'] * aa1['n']
report = pd.merge(report, aa1, on=['exchange', 'sta', 'colo', 'accCode', 'isMsg'], how='left')
assert(report[report['mean'].isnull()].shape[0] == 0)

from IPython.display import display, HTML
display(HTML(report[((report['cur_med'] > report['max']) | (report['cur_med'] < report['min'])) \
                    & (report['cur_count'] > 100) & (abs(report['cur_med'] - report['prev_med']) > 10)] \
             .groupby(['exchange', 'sta', 'colo', 'accCode', 'isMsg'])['prev_med', 'cur_med', 'prev_95p', 'cur_95p'].first().to_html()))
report = report[((report['cur_med'] > report['max']) | (report['cur_med'] < report['min'])) \
                    & (report['cur_count'] > 100) & (abs(report['cur_med'] - report['prev_med']) > 10)] \
             .groupby(['exchange', 'sta', 'colo', 'accCode', 'isMsg'])['prev_med', 'cur_med', 'prev_95p', 'cur_95p'].first().reset_index()

body += '<div>In the following cases, internal latency under given accCode pass the hurdle we set:'
result = [report['exchange'].values, report['sta'].values, report['colo'].values, report['accCode'].values, report['isMsg'].values, 
         report['prev_med'].values, report['prev_95p'].values, report['cur_med'].values, report['cur_95p'].values]
title = ['exchange', 'sta', 'colo', 'accCode', 'isMsg', 'prev_med', 'prev_95p', 'cur_med', 'cur_95p']
body += convertToHtml(result,title)

410

410

### 3. tickToMBD

In [5]:
body += '<p><b>IV. tickToMBD</b></p>'

import os
import glob
import datetime
import numpy as np
import pandas as pd

startDate = str(orderLog['date'].min())
endDate = str(orderLog['date'].max())
readPath = r'L:\orderLog\result\marketPos'
dataPathLs = np.array(glob.glob(os.path.join(readPath, 'SZspeed_***.pkl')))
dateLs = np.array([os.path.basename(i).split('.')[0].split('_')[1] for i in dataPathLs])
dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]
checkData = []
for path in dataPathLs:
    data = pd.read_pickle(path)
    checkData += [data]
checkLog = pd.concat(checkData).reset_index(drop=True)   
checkLog['date'].unique()

checkLog = checkLog.drop_duplicates(['date', 'secid', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'], keep=False)
checkLog = checkLog[~checkLog['accCode'].isnull()]
checkLog['tag'] = 'previous'
checkLog.loc[checkLog['date'] == orderLog['date'].max(), 'tag'] = 'current'

cc1 = checkLog
cc1 = cc1.reset_index(drop=True)
cc1['ordering'] = cc1.index
cc1['time_diff'] = cc1['clockAtArrival'] - cc1['start_time']
cc1['colo1'] = cc1['colo'].str[:2] + cc1['colo'].str[3:5] + cc1['colo'].str[6:8]
cc1['colo_broker'] = cc1['colo1'] + '_' + cc1["accCode"].astype(int).astype(str)
cc1['accCode'] = cc1['accCode'].fillna(0).astype(int)
cc1['sta'] = np.where(cc1['mrstaat'] == 1000, 'staone', np.where(
cc1['mrstaat'] == 3000, 'statwo', 'sta300'))


re1 = cc1.groupby(['tag', 'sta', 'colo', 'accCode'])['time_diff'].describe().fillna(0).astype(int).reset_index()
# re1 = re1[re1['count'] > 20].reset_index()
c1 = cc1.groupby(['tag', 'sta', 'colo', 'accCode'])['time_diff'].apply(lambda x: x.describe([0.1])['10%']).astype(int).reset_index()
c1 = c1.rename(columns={"time_diff":"10%"})
re1 = pd.merge(re1, c1[['tag', 'sta', 'colo', 'accCode', '10%']], on=['tag', 'sta', 'colo', 'accCode'])
c1 = cc1.groupby(['tag', 'sta', 'colo', 'accCode'])['time_diff'].apply(lambda x: x.describe([0.9])['90%']).astype(int).reset_index()
c1 = c1.rename(columns={"time_diff":"90%"})
re1 = pd.merge(re1, c1[['tag', 'sta', 'colo', 'accCode', '90%']], on=['tag', 'sta', 'colo', 'accCode'])
ree1 = re1[re1['tag'] == 'previous']
ree1 = ree1.rename(columns={'50%':'prev_med', 'count':'prev_count'})
ree2 = re1[re1['tag'] == 'current']
ree2 = ree2.rename(columns={'50%':'cur_med', 'count':'cur_count'})
report = pd.merge(ree1[['sta', 'colo', 'accCode', 'prev_count', 'prev_med']], 
               ree2[['sta', 'colo', 'accCode', 'cur_count', 'cur_med']], on=['sta', 'colo', 'accCode'])


aa = cc1[(cc1['updateType'] == 0) & (cc1['tag'] == 'previous')].groupby(['date', 'sta', 'colo', 'accCode'])['time_diff'].median().reset_index()
aa['count'] = aa.groupby(['sta', 'colo', 'accCode'])['date'].transform('nunique')
aa1 = aa.groupby(['sta', 'colo', 'accCode'])['time_diff'].describe()[['mean', 'std']].reset_index()
aa1 = aa1.fillna(0)
aa = pd.merge(aa, aa1, on=['sta', 'colo', 'accCode'])
for j in [1]:
    aa[str(j) + 'std_low'] = aa['mean'] - j*aa['std']
    aa[str(j) + 'std_high'] = aa['mean'] + j*aa['std']
    aa['count1'] = np.where((aa['time_diff'] <= aa[str(j) + 'std_high']) & (aa['time_diff'] >= aa[str(j) + 'std_low']), 1, 0)
    re1 = aa.groupby(['sta', 'colo', 'accCode'])['count1'].sum().reset_index()
    re2 = aa.groupby(['sta', 'colo', 'accCode'])['count'].first().reset_index()
    re1 = pd.merge(re2, re1, on=['sta', 'colo', 'accCode'])
    re1['count1'] = re1['count1'] / re1['count']
    re1 = re1.rename(columns={'count1':str(j)+'*std'})
for j in [1.5, 2, 2.5, 3, 3.5, 4, 4.5]:
    aa[str(j) + 'std_low'] = aa['mean'] - j*aa['std']
    aa[str(j) + 'std_high'] = aa['mean'] + j*aa['std']
    aa['count1'] = np.where((aa['time_diff'] <= aa[str(j) + 'std_high']) & (aa['time_diff'] >= aa[str(j) + 'std_low']), 1, 0)
    re = aa.groupby(['sta', 'colo', 'accCode'])['count1'].sum().reset_index()
    re2 = aa.groupby(['sta', 'colo', 'accCode'])['count'].first().reset_index()
    re = pd.merge(re2, re, on=['sta', 'colo', 'accCode'])
    re['count1'] = re['count1'] / re['count']
    re = re.rename(columns={'count1':str(j)+'*std'})
    re1 = pd.merge(re1, re[['sta', 'colo', 'accCode', str(j)+'*std']], on=['sta', 'colo', 'accCode'])
nc = []
for i in range(0, re1.shape[0]):
    nc.append(np.float(re1.columns[6:][(re1.iloc[i, 6:] == 1)][0].split('*')[0]))
re1['n'] = nc
display(re1.shape[0])
display(aa1.shape[0])
aa1 = pd.merge(aa1, re1[['sta', 'colo', 'accCode', 'n']], on=['sta', 'colo', 'accCode'])
aa1['min'] = aa1['mean'] - aa1['std'] * aa1['n']
aa1['max'] = aa1['mean'] + aa1['std'] * aa1['n']
report = pd.merge(report, aa1, on=['sta', 'colo', 'accCode'], how='left')
assert(report[report['mean'].isnull()].shape[0] == 0)

from IPython.display import display, HTML
display(HTML(report[((report['cur_med'] > report['max']) | (report['cur_med'] < report['min'])) \
                    & (report['cur_count'] > 100) & (abs(report['cur_med'] - report['prev_med']) > 1000)] \
             .groupby(['sta', 'colo', 'accCode'])['prev_med', 'cur_med'].first().to_html()))
report = report[((report['cur_med'] > report['max']) | (report['cur_med'] < report['min'])) \
                    & (report['cur_count'] > 100) & (abs(report['cur_med'] - report['prev_med']) > 1000)] \
             .groupby(['sta', 'colo', 'accCode'])['prev_med', 'cur_med'].first().reset_index()
body += '<div>In the following cases, tickToMBD under given accCode pass the hurdle we set:<div>'
result = [report['sta'].values, report['colo'].values, report['accCode'].values,
         report['prev_med'].values, report['cur_med'].values]
title = ['sta', 'colo', 'accCode', 'prev_med', 'cur_med']
body += convertToHtml(result,title)

113

113

,,,prev_med,cur_med
sta,colo,accCode,,
statwo,zs_52_08,5377,5001,3792


In [6]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

def sendMail(title, body):
    smtp_server = '42.120.226.4' # 'smtp.mxhichina.com'
    user = 'zhenyu.yin@general-int.com'
    passwd = 'Yqzy0063!'
    from_addr = 'zhenyu.yin@general-int.com'
    to_addr = ['zhenyu.yin@general-int.com']

    msg = MIMEMultipart()
    msg['From'] = from_addr
    msg['To'] = ', '.join(to_addr)
    msg['Subject'] = title
    txt = MIMEText(body, _subtype='html', _charset='UTF-8')
    msg.attach(txt)

    smtp = None
    while True:
        try:
            smtp = smtplib.SMTP(smtp_server)
            print('smtp server connected')
            smtp.login(user, passwd)
            print('login')
            break
        except Exception as e:
            print(e)
    print('send mail')
    smtp.sendmail(from_addr, to_addr, msg.as_string())
    smtp.quit()

sendMail(str(orderLog['date'].max()) + ' daily report', body + "</div></body></html>")

smtp server connected
login
send mail
